In [57]:
import torch
import scipy.io as io
import plotly.offline as py
import plotly.graph_objs as go
import json
import numpy as np
from sklearn.svm import SVC
from sklearn import metrics

import pickle 

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import mean_squared_error
import math as m

py.init_notebook_mode(connected=True)


In [58]:
# Loading Matlab Struct array with all of the data:

mat = io.loadmat('data_Mg_GBperatom_seg_2Al_dump.mat')

length_A = mat['A'].shape[1]

In [59]:
# Organizing data:

for i in range(30):
    segE = mat['A']['Eseg'][0,i]
    #check whether this is a valid data?
    n1 = segE[:,0] != 0 
    segE = np.squeeze(segE[n1,:])
    atom_ID = segE[:,0].astype(int) - 1

    descriptor = mat['A']['peratom'][0,i][0,0]
    descriptor_temp = np.concatenate([descriptor['pos'],descriptor['pe'],descriptor['cna'],descriptor['centro_fnn'],
                                descriptor['centro_snn'],descriptor['coord'],descriptor['f'],descriptor['stress'],
                                descriptor['voronoi']], axis = 1)
    if i == 0:
        descriptor_all = descriptor_temp[atom_ID]
        segE_all = segE
    else:
        descriptor_temp = descriptor_temp[atom_ID]
        descriptor_all = np.concatenate([descriptor_all, descriptor_temp], axis = 0)
        segE_all = np.concatenate([segE_all, segE])

descriptor_all[:,2] = abs(descriptor_all[:,2]-min(descriptor_all[:,2])-20)
sigma_H = np.sum(descriptor_all[:,11:14], axis = 1)/3
f_mag = np.linalg.norm(descriptor_all[:,8:11], axis = 1, ord = 2)

feature = np.concatenate([descriptor_all, sigma_H[:,np.newaxis], f_mag[:,np.newaxis]], axis = 1)

In [60]:
# Neural Network

y_true = segE_all[:,1]    # y_true= MD segregation energies

pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]

# Separation of each descriptor (feature):
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Removal of least important features below 
# The optimal combination of features to use was found using 
# Recursive Feature Elimination. This was applied
# to each base estimator (see bottom of file for more info):
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)

n= feature1.shape[1]
feature_space = feature1


#random shuffle
np.random.seed(10)   #Randomizing Seed
idx0 = np.random.permutation(np.arange(len(feature_space)))  
feature_space = feature_space[idx0]
y_true = y_true[idx0]

# #divide data into training, crossvalildation and test sets:
# X_train = feature_space[0:int(len(feature_space)*.7)]
# X_mean = np.mean(X_train, axis = 0)
# X_std = np.std(X_train, axis = 0)
# X_train = (X_train - X_mean)/X_std
# X_temp = feature_space[int(len(feature_space)*.7):]
# X_temp = (X_temp - X_mean)/X_std
# X_test = X_temp[:int(len(X_temp)*.5)]
# X_cos = X_temp[int(len(X_temp)*.5):]

# Y_train = y_true[0:int(len(feature_space)*.7)]
# Y_mean = np.mean(Y_train, axis = 0)
# Y_std = np.std(Y_train, axis = 0)
# Y_temp = y_true[int(len(feature_space)*.7):]
# Y_test = Y_temp[:int(len(X_temp)*.5)]
# Y_cos = Y_temp[int(len(X_temp)*.5):]





X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

X_fold1 = (X_fold1 - X_mean1)/X_std1
X_fold2 = (X_fold2 - X_mean2)/X_std2
X_fold3 = (X_fold3 - X_mean3)/X_std3
X_fold4 = (X_fold4 - X_mean4)/X_std4
X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


print('-----------------------------','\n')


(2846, 15)
(2846, 15)
(2846, 15)
(2846, 15)
(2847, 15) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 

----------------------------- 



In [61]:
R2_train_list =[]
R2_val_list = []

In [62]:



torch.manual_seed(10)


(N, D_in) = X_train1.shape
H1 = 200
H2 = 50
H3 = 25
D_out = 1

print(D_in)
#print(H)

X_train1 = torch.from_numpy(X_train1)
X_train1 = X_train1.float()
y = torch.from_numpy(y_train1)
y = y.float()
y = y.view(y.shape[0],1)
modelnnbcv1 = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out)
)

#model1.apply(weights_init)     # initialize weights

loss_fn2 = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4    #orig: 1e-4

#optimizer2 = torch.optim.SGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, momentum = 0.9)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.Adagrad(modelnn.parameters(), lr= learning_rate, lr_decay= 0.1, weight_decay=0.3, initial_accumulator_value=0)#, eps= 1e-10)
optimizer2= torch.optim.RMSprop(modelnnbcv1.parameters(), lr = learning_rate, alpha=0.85, eps=1e-08, weight_decay=0.06, momentum=0.9)
#optimizer2 = torch.optim.Adamax(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, betas=(0.4, 0.999), eps=1e-6)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.ASGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, lambd=5e-1, alpha=0.65, t0=1e7)#, momentum = 0.9)#, weight_decay = 0.3)#0.2


for t in range(10000):
    optimizer2.zero_grad()
    
    y_pred = modelnnbcv1(X_train1)
    
    loss = loss_fn2(y_pred, y)
    if t%1000 == 0:
        print(t, loss.item())
    
    loss.backward()
    
    optimizer2.step()
    



15
0 64.57832336425781
1000 0.4667019844055176
2000 0.44009822607040405
3000 0.4143255949020386
4000 0.3887212574481964
5000 0.34665828943252563
6000 0.3366284966468811
7000 0.31479859352111816
8000 0.3381136953830719
9000 0.3244890868663788


In [63]:
X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


X_train1 = torch.from_numpy(X_train1)
X_train1 = X_train1.float()
Y_train_pred = modelnnbcv1(X_train1)   #
Y_train_pred = Y_train_pred.detach().numpy()
Y_train_pred = Y_train_pred.squeeze()
Y_train = y_train1.squeeze()
Y_mean = np.mean(Y_train)
SS_tot = np.sum(np.power(Y_train - Y_mean, 2))
SS_res = np.sum(np.power(Y_train_pred - Y_train, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared Train:', R_squared)
R2_train_list.append(R_squared)



X_test1 = torch.from_numpy(X_test1)
X_test1 = X_test1.float()
Y_test_pred = modelnnbcv1(X_test1)    #
Y_test_pred = Y_test_pred.detach().numpy()
Y_test_pred = Y_test_pred.squeeze()
Y_test = y_test1.squeeze()
Y_mean = np.mean(Y_test)
SS_tot = np.sum(np.power(Y_test - Y_mean, 2))
SS_res = np.sum(np.power(Y_test_pred - Y_test, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared val:', R_squared)
R2_val_list.append(R_squared)


# X_cos = torch.from_numpy(X_cos)
# X_cos = X_cos.float()
# Y_cos_pred = modelnnb(X_cos)
# Y_cos_pred = Y_cos_pred.detach().numpy()
# Y_cos_pred = Y_cos_pred.squeeze()
# Y_cos = Y_cos.squeeze()
# Y_mean = np.mean(Y_cos)
# SS_tot = np.sum(np.power(Y_cos - Y_mean, 2))
# SS_res = np.sum(np.power(Y_cos_pred - Y_cos, 2))
# R_squared = 1-SS_res/SS_tot

# print('R_squared Cross Validation:', R_squared)





R_squared Train: 0.9849871183215958
R_squared val: 0.9643650124224278


In [64]:
# RMSE Calculation
Y_test_pred_nn = modelnnbcv1(X_test1)
Y_test_pred_nn = Y_test_pred_nn.detach().numpy()
Y_test_pred_nn = Y_test_pred_nn.squeeze()

print('RMSE:', np.sqrt(mean_squared_error(y_test1,Y_test_pred_nn)),'\n')

RMSE: 0.009009650532740512 



In [65]:
# Saving Model

modelnnbcv1_pkl_filename = 'modelnnbcv1.pkl'
modelnnbcv1_pkl = open(modelnnbcv1_pkl_filename, 'wb')
pickle.dump(modelnnbcv1, modelnnbcv1_pkl)
modelnnbcv1_pkl.close()

# Round 2

In [66]:
# Neural Network

y_true = segE_all[:,1]    # y_true= MD segregation energies

pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]

# Separation of each descriptor (feature):
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Removal of least important features below 
# The optimal combination of features to use was found using 
# Recursive Feature Elimination. This was applied
# to each base estimator (see bottom of file for more info):
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)

n= feature1.shape[1]
feature_space = feature1


#random shuffle
np.random.seed(10)   #Randomizing Seed
idx0 = np.random.permutation(np.arange(len(feature_space)))  
feature_space = feature_space[idx0]
y_true = y_true[idx0]

# #divide data into training, crossvalildation and test sets:
# X_train = feature_space[0:int(len(feature_space)*.7)]
# X_mean = np.mean(X_train, axis = 0)
# X_std = np.std(X_train, axis = 0)
# X_train = (X_train - X_mean)/X_std
# X_temp = feature_space[int(len(feature_space)*.7):]
# X_temp = (X_temp - X_mean)/X_std
# X_test = X_temp[:int(len(X_temp)*.5)]
# X_cos = X_temp[int(len(X_temp)*.5):]

# Y_train = y_true[0:int(len(feature_space)*.7)]
# Y_mean = np.mean(Y_train, axis = 0)
# Y_std = np.std(Y_train, axis = 0)
# Y_temp = y_true[int(len(feature_space)*.7):]
# Y_test = Y_temp[:int(len(X_temp)*.5)]
# Y_cos = Y_temp[int(len(X_temp)*.5):]





X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

X_fold1 = (X_fold1 - X_mean1)/X_std1
X_fold2 = (X_fold2 - X_mean2)/X_std2
X_fold3 = (X_fold3 - X_mean3)/X_std3
X_fold4 = (X_fold4 - X_mean4)/X_std4
X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


print('-----------------------------','\n')


(2846, 15)
(2846, 15)
(2846, 15)
(2846, 15)
(2847, 15) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 

----------------------------- 



In [67]:



torch.manual_seed(10)


(N, D_in) = X_train2.shape  #
H1 = 200
H2 = 50
H3 = 25
D_out = 1

print(D_in)
#print(H)

X_train2 = torch.from_numpy(X_train2)  #
X_train2 = X_train2.float()            #
y = torch.from_numpy(y_train2)         #
y = y.float()
y = y.view(y.shape[0],1)
modelnnbcv2 = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out)
)

#model1.apply(weights_init)     # initialize weights

loss_fn2 = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4    #orig: 1e-4

#optimizer2 = torch.optim.SGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, momentum = 0.9)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.Adagrad(modelnn.parameters(), lr= learning_rate, lr_decay= 0.1, weight_decay=0.3, initial_accumulator_value=0)#, eps= 1e-10)
optimizer2= torch.optim.RMSprop(modelnnbcv2.parameters(), lr = learning_rate, alpha=0.85, eps=1e-08, weight_decay=0.06, momentum=0.9)
#optimizer2 = torch.optim.Adamax(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, betas=(0.4, 0.999), eps=1e-6)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.ASGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, lambd=5e-1, alpha=0.65, t0=1e7)#, momentum = 0.9)#, weight_decay = 0.3)#0.2


for t in range(10000):
    optimizer2.zero_grad()
    
    y_pred = modelnnbcv2(X_train2)     #
    
    loss = loss_fn2(y_pred, y)
    if t%1000 == 0:
        print(t, loss.item())
    
    loss.backward()
    
    optimizer2.step()
    





15
0 65.67202758789062
1000 0.479742169380188
2000 0.41044101119041443
3000 0.43867549300193787
4000 0.35980623960494995
5000 0.40635350346565247
6000 0.3795199990272522
7000 0.34898841381073
8000 0.36741122603416443
9000 0.35851922631263733


In [68]:
X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1







X_train2 = torch.from_numpy(X_train2)        #
X_train2 = X_train2.float()                 #
Y_train_pred = modelnnbcv2(X_train2)           #
Y_train_pred = Y_train_pred.detach().numpy()
Y_train_pred = Y_train_pred.squeeze()
Y_train = y_train2.squeeze()                #
Y_mean = np.mean(Y_train)
SS_tot = np.sum(np.power(Y_train - Y_mean, 2))
SS_res = np.sum(np.power(Y_train_pred - Y_train, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared Train:', R_squared)
R2_train_list.append(R_squared)


X_test2 = torch.from_numpy(X_test2)      #
X_test2 = X_test2.float()                #
Y_test_pred = modelnnbcv2(X_test2)         #
Y_test_pred = Y_test_pred.detach().numpy()
Y_test_pred = Y_test_pred.squeeze()
Y_test = y_test2.squeeze()            #
Y_mean = np.mean(Y_test)
SS_tot = np.sum(np.power(Y_test - Y_mean, 2))
SS_res = np.sum(np.power(Y_test_pred - Y_test, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared val:', R_squared)
R2_val_list.append(R_squared)


# X_cos = torch.from_numpy(X_cos)
# X_cos = X_cos.float()
# Y_cos_pred = modelnnb(X_cos)
# Y_cos_pred = Y_cos_pred.detach().numpy()
# Y_cos_pred = Y_cos_pred.squeeze()
# Y_cos = Y_cos.squeeze()
# Y_mean = np.mean(Y_cos)
# SS_tot = np.sum(np.power(Y_cos - Y_mean, 2))
# SS_res = np.sum(np.power(Y_cos_pred - Y_cos, 2))
# R_squared = 1-SS_res/SS_tot

# print('R_squared Cross Validation:', R_squared)





R_squared Train: 0.9855009500880881
R_squared val: 0.9621413513406336


In [69]:
# Saving Model

modelnnbcv2_pkl_filename = 'modelnnbcv2.pkl'
modelnnbcv2_pkl = open(modelnnbcv2_pkl_filename, 'wb')
pickle.dump(modelnnbcv2, modelnnbcv2_pkl)
modelnnbcv2_pkl.close()

# Round 3

In [70]:
# Neural Network

y_true = segE_all[:,1]    # y_true= MD segregation energies

pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]

# Separation of each descriptor (feature):
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Removal of least important features below 
# The optimal combination of features to use was found using 
# Recursive Feature Elimination. This was applied
# to each base estimator (see bottom of file for more info):
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)

n= feature1.shape[1]
feature_space = feature1


#random shuffle
np.random.seed(10)   #Randomizing Seed
idx0 = np.random.permutation(np.arange(len(feature_space)))  
feature_space = feature_space[idx0]
y_true = y_true[idx0]

# #divide data into training, crossvalildation and test sets:
# X_train = feature_space[0:int(len(feature_space)*.7)]
# X_mean = np.mean(X_train, axis = 0)
# X_std = np.std(X_train, axis = 0)
# X_train = (X_train - X_mean)/X_std
# X_temp = feature_space[int(len(feature_space)*.7):]
# X_temp = (X_temp - X_mean)/X_std
# X_test = X_temp[:int(len(X_temp)*.5)]
# X_cos = X_temp[int(len(X_temp)*.5):]

# Y_train = y_true[0:int(len(feature_space)*.7)]
# Y_mean = np.mean(Y_train, axis = 0)
# Y_std = np.std(Y_train, axis = 0)
# Y_temp = y_true[int(len(feature_space)*.7):]
# Y_test = Y_temp[:int(len(X_temp)*.5)]
# Y_cos = Y_temp[int(len(X_temp)*.5):]





X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

X_fold1 = (X_fold1 - X_mean1)/X_std1
X_fold2 = (X_fold2 - X_mean2)/X_std2
X_fold3 = (X_fold3 - X_mean3)/X_std3
X_fold4 = (X_fold4 - X_mean4)/X_std4
X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


print('-----------------------------','\n')


(2846, 15)
(2846, 15)
(2846, 15)
(2846, 15)
(2847, 15) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 

----------------------------- 



In [71]:



torch.manual_seed(10)


(N, D_in) = X_train3.shape  #
H1 = 200
H2 = 50
H3 = 25
D_out = 1

print(D_in)
#print(H)

X_train3 = torch.from_numpy(X_train3)  #
X_train3 = X_train3.float()            #
y = torch.from_numpy(y_train3)         #
y = y.float()
y = y.view(y.shape[0],1)
modelnnbcv3 = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out)
)

#model1.apply(weights_init)     # initialize weights

loss_fn2 = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4    #orig: 1e-4

#optimizer2 = torch.optim.SGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, momentum = 0.9)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.Adagrad(modelnn.parameters(), lr= learning_rate, lr_decay= 0.1, weight_decay=0.3, initial_accumulator_value=0)#, eps= 1e-10)
optimizer2= torch.optim.RMSprop(modelnnbcv3.parameters(), lr = learning_rate, alpha=0.85, eps=1e-08, weight_decay=0.06, momentum=0.9)
#optimizer2 = torch.optim.Adamax(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, betas=(0.4, 0.999), eps=1e-6)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.ASGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, lambd=5e-1, alpha=0.65, t0=1e7)#, momentum = 0.9)#, weight_decay = 0.3)#0.2


for t in range(10000):
    optimizer2.zero_grad()
    
    y_pred = modelnnbcv3(X_train3)       ##
    
    loss = loss_fn2(y_pred, y)
    if t%1000 == 0:
        print(t, loss.item())
    
    loss.backward()
    
    optimizer2.step()
    





15
0 65.6483383178711
1000 0.4732404351234436
2000 0.4563058316707611
3000 0.38391849398612976
4000 0.3626531660556793
5000 0.4006459414958954
6000 0.3847166895866394
7000 0.3775559365749359
8000 0.3329657018184662
9000 0.32290950417518616


In [72]:
X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


X_train3 = torch.from_numpy(X_train3)        #
X_train3 = X_train3.float()                 #
Y_train_pred = modelnnbcv3(X_train3)           #
Y_train_pred = Y_train_pred.detach().numpy()
Y_train_pred = Y_train_pred.squeeze()
Y_train = y_train3.squeeze()                #
Y_mean = np.mean(Y_train)
SS_tot = np.sum(np.power(Y_train - Y_mean, 2))
SS_res = np.sum(np.power(Y_train_pred - Y_train, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared Train:', R_squared)
R2_train_list.append(R_squared)


X_test3 = torch.from_numpy(X_test3)      #
X_test3 = X_test3.float()                #
Y_test_pred = modelnnbcv3(X_test3)         #
Y_test_pred = Y_test_pred.detach().numpy()
Y_test_pred = Y_test_pred.squeeze()
Y_test = y_test3.squeeze()            #
Y_mean = np.mean(Y_test)
SS_tot = np.sum(np.power(Y_test - Y_mean, 2))
SS_res = np.sum(np.power(Y_test_pred - Y_test, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared val:', R_squared)
R2_val_list.append(R_squared)


# X_cos = torch.from_numpy(X_cos)
# X_cos = X_cos.float()
# Y_cos_pred = modelnnb(X_cos)
# Y_cos_pred = Y_cos_pred.detach().numpy()
# Y_cos_pred = Y_cos_pred.squeeze()
# Y_cos = Y_cos.squeeze()
# Y_mean = np.mean(Y_cos)
# SS_tot = np.sum(np.power(Y_cos - Y_mean, 2))
# SS_res = np.sum(np.power(Y_cos_pred - Y_cos, 2))
# R_squared = 1-SS_res/SS_tot

# print('R_squared Cross Validation:', R_squared)





R_squared Train: 0.9834642996058409
R_squared val: 0.960232123313809


In [73]:
# # Saving Model

# modelnnbcv3_pkl_filename = 'modelnnbcv3.pkl'
# modelnnbcv3_pkl = open(modelnnbcv3_pkl_filename, 'wb')
# pickle.dump(modelnnbcv3, modelnnbcv3_pkl)
# modelnnbcv3_pkl.close()

# Round 4

In [74]:
# Neural Network

y_true = segE_all[:,1]    # y_true= MD segregation energies

pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]

# Separation of each descriptor (feature):
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Removal of least important features below 
# The optimal combination of features to use was found using 
# Recursive Feature Elimination. This was applied
# to each base estimator (see bottom of file for more info):
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)

n= feature1.shape[1]
feature_space = feature1


#random shuffle
np.random.seed(10)   #Randomizing Seed
idx0 = np.random.permutation(np.arange(len(feature_space)))  
feature_space = feature_space[idx0]
y_true = y_true[idx0]

# #divide data into training, crossvalildation and test sets:
# X_train = feature_space[0:int(len(feature_space)*.7)]
# X_mean = np.mean(X_train, axis = 0)
# X_std = np.std(X_train, axis = 0)
# X_train = (X_train - X_mean)/X_std
# X_temp = feature_space[int(len(feature_space)*.7):]
# X_temp = (X_temp - X_mean)/X_std
# X_test = X_temp[:int(len(X_temp)*.5)]
# X_cos = X_temp[int(len(X_temp)*.5):]

# Y_train = y_true[0:int(len(feature_space)*.7)]
# Y_mean = np.mean(Y_train, axis = 0)
# Y_std = np.std(Y_train, axis = 0)
# Y_temp = y_true[int(len(feature_space)*.7):]
# Y_test = Y_temp[:int(len(X_temp)*.5)]
# Y_cos = Y_temp[int(len(X_temp)*.5):]





X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

X_fold1 = (X_fold1 - X_mean1)/X_std1
X_fold2 = (X_fold2 - X_mean2)/X_std2
X_fold3 = (X_fold3 - X_mean3)/X_std3
X_fold4 = (X_fold4 - X_mean4)/X_std4
X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


print('-----------------------------','\n')


(2846, 15)
(2846, 15)
(2846, 15)
(2846, 15)
(2847, 15) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 

----------------------------- 



In [75]:



torch.manual_seed(10)


(N, D_in) = X_train4.shape  #
H1 = 200
H2 = 50
H3 = 25
D_out = 1

print(D_in)
#print(H)

X_train4 = torch.from_numpy(X_train4)  #
X_train4 = X_train4.float()            #
y = torch.from_numpy(y_train4)         #
y = y.float()
y = y.view(y.shape[0],1)
modelnnbcv4 = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out)
)

#model1.apply(weights_init)     # initialize weights

loss_fn2 = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4    #orig: 1e-4

#optimizer2 = torch.optim.SGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, momentum = 0.9)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.Adagrad(modelnn.parameters(), lr= learning_rate, lr_decay= 0.1, weight_decay=0.3, initial_accumulator_value=0)#, eps= 1e-10)
optimizer2= torch.optim.RMSprop(modelnnbcv4.parameters(), lr = learning_rate, alpha=0.85, eps=1e-08, weight_decay=0.06, momentum=0.9)
#optimizer2 = torch.optim.Adamax(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, betas=(0.4, 0.999), eps=1e-6)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.ASGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, lambd=5e-1, alpha=0.65, t0=1e7)#, momentum = 0.9)#, weight_decay = 0.3)#0.2


for t in range(10000):
    optimizer2.zero_grad()
    
    y_pred = modelnnbcv4(X_train4)       ##
    
    loss = loss_fn2(y_pred, y)
    if t%1000 == 0:
        print(t, loss.item())
    
    loss.backward()
    
    optimizer2.step()
    





15
0 65.95205688476562
1000 0.5195485353469849
2000 0.4277627170085907
3000 0.4133763313293457
4000 0.3999062180519104
5000 0.419999361038208
6000 0.38540974259376526
7000 0.3319950997829437
8000 0.3407895565032959
9000 0.3234460651874542


In [76]:
X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


X_train4 = torch.from_numpy(X_train4)        #
X_train4 = X_train4.float()                 #
Y_train_pred = modelnnbcv4(X_train4)           #
Y_train_pred = Y_train_pred.detach().numpy()
Y_train_pred = Y_train_pred.squeeze()
Y_train = y_train4.squeeze()                #
Y_mean = np.mean(Y_train)
SS_tot = np.sum(np.power(Y_train - Y_mean, 2))
SS_res = np.sum(np.power(Y_train_pred - Y_train, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared Train:', R_squared)
R2_train_list.append(R_squared)



X_test4 = torch.from_numpy(X_test4)      #
X_test4 = X_test4.float()                #
Y_test_pred = modelnnbcv4(X_test4)         #
Y_test_pred = Y_test_pred.detach().numpy()
Y_test_pred = Y_test_pred.squeeze()
Y_test = y_test4.squeeze()            #
Y_mean = np.mean(Y_test)
SS_tot = np.sum(np.power(Y_test - Y_mean, 2))
SS_res = np.sum(np.power(Y_test_pred - Y_test, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared val:', R_squared)
R2_val_list.append(R_squared)


# X_cos = torch.from_numpy(X_cos)
# X_cos = X_cos.float()
# Y_cos_pred = modelnnb(X_cos)
# Y_cos_pred = Y_cos_pred.detach().numpy()
# Y_cos_pred = Y_cos_pred.squeeze()
# Y_cos = Y_cos.squeeze()
# Y_mean = np.mean(Y_cos)
# SS_tot = np.sum(np.power(Y_cos - Y_mean, 2))
# SS_res = np.sum(np.power(Y_cos_pred - Y_cos, 2))
# R_squared = 1-SS_res/SS_tot

# print('R_squared Cross Validation:', R_squared)





R_squared Train: 0.9860265157902042
R_squared val: 0.9529886346302084


In [77]:
# # Saving Model

# modelnnbcv4_pkl_filename = 'modelnnbcv4.pkl'
# modelnnbcv4_pkl = open(modelnnbcv4_pkl_filename, 'wb')
# pickle.dump(modelnnbcv4, modelnnbcv4_pkl)
# modelnnbcv4_pkl.close()

# Round 5

In [78]:
# Neural Network

y_true = segE_all[:,1]    # y_true= MD segregation energies

pos = feature[:,:3]
zpos = pos[:,2][:,np.newaxis]
ypos= pos[:,1][:,np.newaxis]
xpos= pos[:,0][:,np.newaxis]

feature1 = feature[:,3:]

# Separation of each descriptor (feature):
f0= feature1[:,0][:,np.newaxis]
f1= feature1[:,1][:,np.newaxis]
f2= feature1[:,2][:,np.newaxis]
f3= feature1[:,3][:,np.newaxis]
f4= feature1[:,4][:,np.newaxis]
f5= feature1[:,5][:,np.newaxis]
f6= feature1[:,6][:,np.newaxis]
f7= feature1[:,7][:,np.newaxis]
f8= feature1[:,8][:,np.newaxis]
f9= feature1[:,9][:,np.newaxis]
f10= feature1[:,10][:,np.newaxis]
f11= feature1[:,11][:,np.newaxis]
f12= feature1[:,12][:,np.newaxis]
f13= feature1[:,13][:,np.newaxis]
f14= feature1[:,14][:,np.newaxis]
f15= feature1[:,15][:,np.newaxis]
f16= feature1[:,16][:,np.newaxis]
f17= feature1[:,17][:,np.newaxis]

# Removal of least important features below 
# The optimal combination of features to use was found using 
# Recursive Feature Elimination. This was applied
# to each base estimator (see bottom of file for more info):
feature1 = np.concatenate([f0,f1,f2,f3,f4,f8,f9,f10,f11,f13,f14,f15,f16,xpos,ypos], axis=1)

n= feature1.shape[1]
feature_space = feature1


#random shuffle
np.random.seed(10)   #Randomizing Seed
idx0 = np.random.permutation(np.arange(len(feature_space)))  
feature_space = feature_space[idx0]
y_true = y_true[idx0]

# #divide data into training, crossvalildation and test sets:
# X_train = feature_space[0:int(len(feature_space)*.7)]
# X_mean = np.mean(X_train, axis = 0)
# X_std = np.std(X_train, axis = 0)
# X_train = (X_train - X_mean)/X_std
# X_temp = feature_space[int(len(feature_space)*.7):]
# X_temp = (X_temp - X_mean)/X_std
# X_test = X_temp[:int(len(X_temp)*.5)]
# X_cos = X_temp[int(len(X_temp)*.5):]

# Y_train = y_true[0:int(len(feature_space)*.7)]
# Y_mean = np.mean(Y_train, axis = 0)
# Y_std = np.std(Y_train, axis = 0)
# Y_temp = y_true[int(len(feature_space)*.7):]
# Y_test = Y_temp[:int(len(X_temp)*.5)]
# Y_cos = Y_temp[int(len(X_temp)*.5):]





X_fold1 = feature_space[0:int(len(feature_space)*.2)]
X_fold2 = feature_space[int(len(feature_space)*.2):int(len(feature_space)*.4)]
X_fold3 = feature_space[int(len(feature_space)*.4):int(len(feature_space)*.6)]
X_fold4 = feature_space[int(len(feature_space)*.6):int(len(feature_space)*.8)]
X_fold5 = feature_space[int(len(feature_space)*.8):]

X_std1 = np.std(X_fold1, axis = 0)
X_std2 = np.std(X_fold2, axis = 0)
X_std3 = np.std(X_fold3, axis = 0)
X_std4 = np.std(X_fold4, axis = 0)
X_std5 = np.std(X_fold5, axis = 0)

X_mean1 = np.mean(X_fold1, axis = 0)
X_mean2 = np.mean(X_fold2, axis = 0)
X_mean3 = np.mean(X_fold3, axis = 0)
X_mean4 = np.mean(X_fold4, axis = 0)
X_mean5 = np.mean(X_fold5, axis = 0)

X_fold1 = (X_fold1 - X_mean1)/X_std1
X_fold2 = (X_fold2 - X_mean2)/X_std2
X_fold3 = (X_fold3 - X_mean3)/X_std3
X_fold4 = (X_fold4 - X_mean4)/X_std4
X_fold5 = (X_fold5 - X_mean5)/X_std5


y_fold1 = y_true[0:int(len(feature_space)*.2)]
y_fold2 = y_true[int(len(feature_space)*.2):int(len(feature_space)*.4)]
y_fold3 = y_true[int(len(feature_space)*.4):int(len(feature_space)*.6)]
y_fold4 = y_true[int(len(feature_space)*.6):int(len(feature_space)*.8)]
y_fold5 = y_true[int(len(feature_space)*.8):]



print(X_fold1.shape)
print(X_fold2.shape)
print(X_fold3.shape)
print(X_fold4.shape)
print(X_fold5.shape,'\n')

print(y_fold1.shape)
print(y_fold2.shape)
print(y_fold3.shape)
print(y_fold4.shape)
print(y_fold5.shape, '\n')



X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


print('-----------------------------','\n')


(2846, 15)
(2846, 15)
(2846, 15)
(2846, 15)
(2847, 15) 

(2846,)
(2846,)
(2846,)
(2846,)
(2847,) 

----------------------------- 



In [79]:



torch.manual_seed(10)


(N, D_in) = X_train5.shape  #
H1 = 200
H2 = 50
H3 = 25
D_out = 1

print(D_in)
#print(H)

X_train5 = torch.from_numpy(X_train5)  #
X_train5 = X_train5.float()            #
y = torch.from_numpy(y_train5)         #
y = y.float()
y = y.view(y.shape[0],1)
modelnnbcv5 = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.ReLU(),
    torch.nn.Linear(H3, D_out)
)

#model1.apply(weights_init)     # initialize weights

loss_fn2 = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4    #orig: 1e-4

#optimizer2 = torch.optim.SGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, momentum = 0.9)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.Adagrad(modelnn.parameters(), lr= learning_rate, lr_decay= 0.1, weight_decay=0.3, initial_accumulator_value=0)#, eps= 1e-10)
optimizer2= torch.optim.RMSprop(modelnnbcv5.parameters(), lr = learning_rate, alpha=0.85, eps=1e-08, weight_decay=0.06, momentum=0.9)
#optimizer2 = torch.optim.Adamax(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, betas=(0.4, 0.999), eps=1e-6)#, weight_decay = 0.3)#0.2
#optimizer2 = torch.optim.ASGD(modelnn.parameters(), lr = learning_rate, weight_decay=0.3, lambd=5e-1, alpha=0.65, t0=1e7)#, momentum = 0.9)#, weight_decay = 0.3)#0.2


for t in range(10000):
    optimizer2.zero_grad()
    
    y_pred = modelnnbcv5(X_train5)       ##
    
    loss = loss_fn2(y_pred, y)
    if t%1000 == 0:
        print(t, loss.item())
    
    loss.backward()
    
    optimizer2.step()
    





15
0 65.33096313476562
1000 0.4826228618621826
2000 0.41234856843948364
3000 0.42075130343437195
4000 0.36585333943367004
5000 0.3703826069831848
6000 0.3643879294395447
7000 0.3803090751171112
8000 0.3679853081703186
9000 0.3438001275062561


In [80]:
X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


X_train5 = torch.from_numpy(X_train5)        #
X_train5 = X_train5.float()                 #
Y_train_pred = modelnnbcv5(X_train5)           #
Y_train_pred = Y_train_pred.detach().numpy()
Y_train_pred = Y_train_pred.squeeze()
Y_train = y_train5.squeeze()                #
Y_mean = np.mean(Y_train)
SS_tot = np.sum(np.power(Y_train - Y_mean, 2))
SS_res = np.sum(np.power(Y_train_pred - Y_train, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared Train:', R_squared)
R2_train_list.append(R_squared)



X_test5 = torch.from_numpy(X_test5)      #
X_test5 = X_test5.float()                #
Y_test_pred = modelnnbcv5(X_test5)         #
Y_test_pred = Y_test_pred.detach().numpy()
Y_test_pred = Y_test_pred.squeeze()
Y_test = y_test5.squeeze()            #
Y_mean = np.mean(Y_test)
SS_tot = np.sum(np.power(Y_test - Y_mean, 2))
SS_res = np.sum(np.power(Y_test_pred - Y_test, 2))
R_squared = 1-SS_res/SS_tot

print('R_squared val:', R_squared)
R2_val_list.append(R_squared)


# X_cos = torch.from_numpy(X_cos)
# X_cos = X_cos.float()
# Y_cos_pred = modelnnb(X_cos)
# Y_cos_pred = Y_cos_pred.detach().numpy()
# Y_cos_pred = Y_cos_pred.squeeze()
# Y_cos = Y_cos.squeeze()
# Y_mean = np.mean(Y_cos)
# SS_tot = np.sum(np.power(Y_cos - Y_mean, 2))
# SS_res = np.sum(np.power(Y_cos_pred - Y_cos, 2))
# R_squared = 1-SS_res/SS_tot

# print('R_squared Cross Validation:', R_squared)





R_squared Train: 0.9843624145316054
R_squared val: 0.9650318362239174


In [81]:
# # Saving Model

# modelnnbcv5_pkl_filename = 'modelnnbcv5.pkl'
# modelnnbcv5_pkl = open(modelnnbcv5_pkl_filename, 'wb')
# pickle.dump(modelnnbcv5, modelnnbcv5_pkl)
# modelnnbcv5_pkl.close()

In [82]:
X_train1 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold4], axis=0)
X_test1 = X_fold5
y_train1 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold4], axis=0)
y_test1 = y_fold5

X_train2 = np.concatenate([X_fold1,X_fold2,X_fold3,X_fold5], axis=0)
X_test2= X_fold4
y_train2 = np.concatenate([y_fold1,y_fold2,y_fold3,y_fold5], axis=0)
y_test2 = y_fold4

X_train3 = np.concatenate([X_fold1,X_fold2,X_fold4,X_fold5], axis=0)
X_test3 = X_fold3
y_train3 = np.concatenate([y_fold1,y_fold2,y_fold4,y_fold5], axis=0)
y_test3 = y_fold3

X_train4 = np.concatenate([X_fold1,X_fold3,X_fold4,X_fold5], axis=0)
X_test4 = X_fold2
y_train4 = np.concatenate([y_fold1,y_fold3,y_fold4,y_fold5], axis=0)
y_test4 = y_fold2

X_train5 = np.concatenate([X_fold2,X_fold3,X_fold4,X_fold5], axis=0)
X_test5 = X_fold1
y_train5 = np.concatenate([y_fold2,y_fold3,y_fold4,y_fold5], axis=0)
y_test5 = y_fold1


In [83]:
# RMSE Calculation 1

X_test1 = torch.from_numpy(X_test1)
X_test1 = X_test1.float()
Y_test_pred_nn = modelnnbcv2(X_test1)
Y_test_pred_nn = Y_test_pred_nn.detach().numpy()
Y_test_pred_nn = Y_test_pred_nn.squeeze()

print('RMSE:', np.sqrt(mean_squared_error(y_test1,Y_test_pred_nn)),'\n')

RMSE: 0.005582242943323989 



In [84]:
# RMSE Calculation 2

X_test2 = torch.from_numpy(X_test2)
X_test2 = X_test2.float()
Y_test_pred_nn = modelnnbcv2(X_test2)
Y_test_pred_nn = Y_test_pred_nn.detach().numpy()
Y_test_pred_nn = Y_test_pred_nn.squeeze()

print('RMSE:', np.sqrt(mean_squared_error(y_test2,Y_test_pred_nn)),'\n')

RMSE: 0.008675287690438323 



In [85]:
# RMSE Calculation 3

X_test3 = torch.from_numpy(X_test3)
X_test3 = X_test3.float()
Y_test_pred_nn = modelnnbcv3(X_test3)
Y_test_pred_nn = Y_test_pred_nn.detach().numpy()
Y_test_pred_nn = Y_test_pred_nn.squeeze()

print('RMSE:', np.sqrt(mean_squared_error(y_test3,Y_test_pred_nn)),'\n')

RMSE: 0.008931011390962097 



In [86]:
# RMSE Calculation 4

X_test4 = torch.from_numpy(X_test4)
X_test4 = X_test4.float()
Y_test_pred_nn = modelnnbcv4(X_test4)
Y_test_pred_nn = Y_test_pred_nn.detach().numpy()
Y_test_pred_nn = Y_test_pred_nn.squeeze()

print('RMSE:', np.sqrt(mean_squared_error(y_test4,Y_test_pred_nn)),'\n')

RMSE: 0.009561601200919543 



In [87]:
# RMSE Calculation 5

X_test5 = torch.from_numpy(X_test5)
X_test5 = X_test5.float()
Y_test_pred_nn = modelnnbcv5(X_test5)
Y_test_pred_nn = Y_test_pred_nn.detach().numpy()
Y_test_pred_nn = Y_test_pred_nn.squeeze()

print('RMSE:', np.sqrt(mean_squared_error(y_test5,Y_test_pred_nn)),'\n')

RMSE: 0.00862574280501505 



In [94]:
rmse = [0.005582242943323989,0.008675287690438323,0.008931011390962097 , 0.009561601200919543 , 0.00862574280501505 ]
print(np.std(rmse))

0.0013870749622810078


In [89]:
print(R2_train_list)
print(R2_val_list)

[0.9849871183215958, 0.9855009500880881, 0.9834642996058409, 0.9860265157902042, 0.9843624145316054]
[0.9643650124224278, 0.9621413513406336, 0.960232123313809, 0.9529886346302084, 0.9650318362239174]


In [90]:
print(np.std(R2_train_list))
print(np.std(R2_val_list))

0.000892520582393606
0.004327624851117232


### Results

In [91]:
# R_squared_train_list = [0.9872147429362317, 0.9888766333675835, 0.9871230941543926, 0.9893489991094322, 0.9893383223780539]
# R_squared_val_list = [0.9626601024742505, 0.965477491545618, 0.9616423575655719, 0.9593789281882419, 0.9674682981144371]
# rmse_list = [0.009222660053090272,  0.008284236793446318, 0.008284236793446318, 0.00888801712687602, 0.008319811424252305]

# print(R_squared_train_list)
# print(R_squared_val_list)
# print(rmse_list)

In [92]:
# print(np.std(R_squared_train_list))
# print(np.std(R_squared_val_list))
# print(np.std(rmse_list))